In [ ]:
import pandas as pd
import os
from PIL import Image

### **Data importation & visualization**

In [2]:
test_data_dir = "data/ISIC_2019_Test_Input/"
train_data_dir = "data/ISIC_2019_Training_Input/"
img_paths = ["data/ISIC_2019_Test_Input/" + os.listdir(test_data_dir)[i] for i in range(len(test_data_dir))]

In [3]:
index = 0
img = Image.open(img_paths[index])

NameError: name 'Image' is not defined